# `CLOUD.jl` - 2D linear advection example

In [106]:
using CLOUD, OrdinaryDiffEq, TimerOutputs

Define the physical problem

In [107]:
θ = π/4 # wave angle
a = sqrt(2) # wave speed
A = 1.0  # amplitude
L = 1.0  # domain length
k = (2*π/L, 2*π/L)  # wave number
T = 0.5/(a*max(abs(cos(θ)),abs(cos(θ)))) # end time for one period

conservation_law = LinearAdvectionEquation((a*cos(θ),a*sin(θ)))
initial_data = InitialDataSine(A,k)
exact_solution = ExactSolution(conservation_law,initial_data);

Set discretization parameters

In [108]:
M = 4
p = 4
p_map = 4
form = WeakConservationForm(mapping_form=SkewSymmetricMapping(), inviscid_numerical_flux=LaxFriedrichsNumericalFlux(0.0))
strategy = ReferenceOperator()

ode_algorithm = CarpenterKennedy2N54()
dt = 0.1*(L/M)/(a*(2*p+1));
write_interval = floor(Int, T/(dt*10));

Set up a `DGSEM` scheme with LG quadrature on quadrilaterals of degree `p` with `M` edges in each direction

In [109]:
#=
reference_approximation = ReferenceApproximation(
    DGSEM(p), Quad(),
    volume_quadrature_rule=LGQuadrature(),
    facet_quadrature_rule=LGQuadrature(),
    mapping_degree=p_map, N_plot=50,
    operator_algorithm=DefaultOperatorAlgorithm())

reference_approximation = ReferenceApproximation(
    DGMulti(p), Tri(), operator_algorithm=BLASAlgorithm())
=#
reference_approximation = ReferenceApproximation(
    CollapsedModal(p), Tri())

mesh = warp_mesh(uniform_periodic_mesh(
    reference_approximation.reference_element, 
    ((0.0,L),(0.0,L)), (M,M)), 
    reference_approximation.reference_element, 0.2)

spatial_discretization = SpatialDiscretization(mesh, 
    reference_approximation)

results_path = save_project(conservation_law,
     spatial_discretization, initial_data, form, (0.0, T), strategy,
     "results/advection_2d_dgsem/", overwrite=true, clear=true)

"results/advection_2d_dgsem/"

Run the solver

In [110]:
ode_problem_1 = semidiscretize(conservation_law,
    spatial_discretization,
    initial_data, 
    form,
    (0.0, T),
    strategy);

save_solution(ode_problem_1.u0, 0.0, results_path, 0)

CLOUD_reset_timer!()
sol = solve(ode_problem_1, ode_algorithm, adaptive=false, dt=dt,
    save_everystep=false, callback=save_callback(results_path, write_interval))
CLOUD_print_timer()

save_solution(last(sol.u), last(sol.t), results_path, "final")

 ────────────────────────────────────────────────────────────────────────────────
            Thread 1                    Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:            2.50s /  95.8%           2.52GiB /  99.0%    

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 local residual         40.8k    2.07s   86.2%  50.6μs   2.31GiB   92.3%  59.2KiB
   apply operators      40.8k    1.55s   64.5%  37.9μs   1.82GiB   72.7%  46.7KiB
     volume terms       40.8k    1.15s   47.8%  28.1μs   1.51GiB   60.2%  38.7KiB
     facet terms        40.8k    177ms    7.4%  4.33μs    157MiB    6.1%  3.95KiB
     mass matrix solve  40.8k    100ms    4.2%  2.44μs   92.8MiB    3.6%  2.33KiB
   gather ext state     40.8k    130ms    5.4%  3.19μs    181MiB    7.1%  4.53KiB
   eval nodal s

Calculate the integral L<sup>2</sup> error norm and conservation/energy error

In [111]:
error_analysis = ErrorAnalysis(results_path, conservation_law, 
    spatial_discretization)
conservation_analysis = PrimaryConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)
energy_analysis = EnergyConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)

println("L2 error:")
println(analyze(error_analysis, last(sol.u), initial_data, T)...)
println("Conservation (initial/final/diff):")
println(analyze(conservation_analysis)...)
println("Energy (initial/final/diff):")
println(analyze(energy_analysis)...)

L2 error:
0.02090809462239079
Conservation (initial/final/diff):
[4.146167930192332e-7][4.146167931848993e-7][1.6566609195578508e-16]
Energy (initial/final/diff):
[0.1249918923475991][0.12499189178972445][-5.578746492540532e-10]


Write output to VTK file

In [112]:
postprocess_vtk(spatial_discretization, string(results_path,"solution.vtu"), last(sol.u));